In [8]:
# FITE7410 Financial Fraud Analytics First Semester, 2025-2026 -----------------
# Assignnment 1 Exploratory Data Analysis (EDA)
# Student: Choi Chi For 3036384024
# Due Date: 19 Oct, 2025 (Sun) 23:59
# Objective of this assignment:
# 1. Perform data cleaning and preparation
# 2. Explore and visualize the data to identify patterns and trends
# 3. Engineer new features based on domain knowledge or insights from EDA
# 4. Prepare a report summarizing the findings from EDA

# Load necessary libraries -----------------------
library(dplyr)
library(tidyr)
library(ggplot2)
library(corrplot)
library(tidyverse)  

ERROR: Error in library(dplyr): there is no package called ‘dplyr’


In [12]:
# R Functions for Substring Operations

string1 <- "# Load necessary libraries" 
string2 <- "-------------------------------------------------------------------"
full_string <- paste(string1, string2)

cat("Original string:", full_string, "\n")
cat("String length:", nchar(full_string), "\n\n")

# Method 1: substr() - Extract substring by position
left_80_chars <- substr(full_string, 1, 80)
cat("Method 1 - substr(string, 1, 80):\n")
cat("Result:", left_80_chars, "\n")
cat("Length:", nchar(left_80_chars), "\n\n")

# Method 2: substring() - Similar to substr() but vectorized
left_80_chars2 <- substring(full_string, 1, 80)
cat("Method 2 - substring(string, 1, 80):\n")
cat("Result:", left_80_chars2, "\n\n")

# Method 3: Using stringr package (part of tidyverse)
library(stringr)
left_80_chars3 <- str_sub(full_string, 1, 80)
cat("Method 3 - str_sub(string, 1, 80) from stringr:\n")
cat("Result:", left_80_chars3, "\n\n")

# Method 4: Extract from right side (last 20 characters)
right_20_chars <- str_sub(full_string, -20, -1)
cat("Method 4 - str_sub(string, -20, -1) - last 20 chars:\n")
cat("Result:", right_20_chars, "\n")

Original string: # Load necessary libraries ------------------------------------------------------------------- 
String length: 94 

Method 1 - substr(string, 1, 80):
Result: # Load necessary libraries ----------------------------------------------------- 
Length: 80 

String length: 94 

Method 1 - substr(string, 1, 80):
Result: # Load necessary libraries ----------------------------------------------------- 
Length: 80 

Method 2 - substring(string, 1, 80):
Result: # Load necessary libraries ----------------------------------------------------- 

Method 2 - substring(string, 1, 80):
Result: # Load necessary libraries ----------------------------------------------------- 

Method 3 - str_sub(string, 1, 80) from stringr:
Method 3 - str_sub(string, 1, 80) from stringr:


Result: # Load necessary libraries ----------------------------------------------------- 

Method 4 - str_sub(string, -20, -1) - last 20 chars:
Result: -------------------- 
Method 4 - str_sub(string, -20, -1) - last 20 chars:
Result: -------------------- 


In [7]:
# String Concatenation in R - Different Methods

# Method 1: paste() function (most common)
string1 <- "Hello"
string2 <- "World"
result1 <- paste(string1, string2)
cat("Method 1 - paste():", result1, "\n")

# Method 2: paste0() function (no separator)
result2 <- paste0(string1, string2)
cat("Method 2 - paste0():", result2, "\n")

# Method 3: paste() with custom separator
result3 <- paste(string1, string2, sep = "-")
cat("Method 3 - paste() with separator:", result3, "\n")

# Method 4: Concatenating multiple strings
string3 <- "from"
string4 <- "R"
result4 <- paste(string1, string2, string3, string4)
cat("Method 4 - Multiple strings:", result4, "\n")

# Method 5: paste() with collapse (for vectors)
string_vector <- c("apple", "banana", "cherry")
result5 <- paste(string_vector, collapse = ", ")
cat("Method 5 - Collapse vector:", result5, "\n")

# Method 6: sprintf() for formatted strings (like printf)
name <- "John"
age <- 25
result6 <- sprintf("My name is %s and I am %d years old", name, age)
cat("Method 6 - sprintf():", result6, "\n")

# Practical example for your assignment
student_name <- "Choi Chi For"
student_id <- "3036384024"
assignment_info <- paste("Student:", student_name, "ID:", student_id, sep = " ")
cat("Assignment example:", assignment_info, "\n")

Method 1 - paste(): Hello World 
Method 2 - paste0(): HelloWorld 
Method 3 - paste() with separator: Hello-World 
Method 4 - Multiple strings: Hello World from R 
Method 5 - Collapse vector: apple, banana, cherry 
Method 6 - sprintf(): My name is John and I am 25 years old 
Assignment example: Student: Choi Chi For ID: 3036384024 
